# WEB SCRAPING

In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

In [2]:
#choose number of ann to put in the functions
def scraping(n_ann):
    df_info = pd.DataFrame(index=['price', 'locali', 'superficie', 'bagni', 'piano'] )
    df_info
    c =1
    i = 1
    while c < n_ann:

        url = "https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag="+str(i)
        html = urlopen(url)
        soup = BeautifulSoup(html, 'lxml')

        mydivs = soup.findAll("div", {"class": "listing-item_body--content"})

        for a in mydivs:

            ann = "announcement"+ str(c)

            data = a.findAll("li", {"class":"lif__item"})

            if len(data) == 6 and c <= n_ann:
                #print(ann, url)

                try:

                    price = data[0].contents[0].replace("\n", "").replace("€ ","").strip()
                except:
                    price = data[0].contents[0].get_text().replace("\n", "").replace("€ ","").strip()


                locali = data[1].get_text().replace("\xa0","").replace("+", "").replace("locali", "").strip()

                superficie = data[2].get_text().replace("da ","").replace("\xa0m2","").replace("superficie","").strip()

                bagni = data[3].get_text().replace("\xa0","").replace("+", "").replace("bagni", "").strip()

                piano = data[4].get_text().replace("\xa0", "").replace("\n","").replace("T","0").replace("piano","").strip()

                df_info[ann] = [price, locali, superficie,bagni,piano]

                c +=1

            else:
                pass
        i +=1

    return df_info
    
    

In [3]:
scraping(50)

,announcement1,announcement2,announcement3,announcement4,announcement5,announcement6,announcement7,announcement8,announcement9,announcement10,...,announcement41,announcement42,announcement43,announcement44,announcement45,announcement46,announcement47,announcement48,announcement49,announcement50
price,225.000,1.350.000,135.000,249.000,279.000,339.000,699.000,189.000,570.000,215.000,...,850.000,1.650.000,139.000,670.000,539.000,375.000,1.150.000,449.000,450.000,519.000
locali,2,4,2,2,3,2,5,4,5,3,...,5,5,2,5,4,3,5,4,4,4
superficie,50,200,60,75,110,70,174,168,160,67,...,183,340,72,227,140,92,160,120,166,150
bagni,1,2,1,1,1,1,3,3,2,1,...,2,3,1,3,2,1,2,2,2,3
piano,1,2,5,1,0,3,A,0,2,1,...,2,4,1,3,3,1,4,4,4,3


In [ ]:
#To review the "A" value for "piano": do we sobstitute it?